# Multi-GPU Basics

Ejemplo básico de cálculo Multi-GPU usando la librería TensorFlow.

Este tutorial requiere que su máquina tenga 2 GPUs
"/cpu:0": La CPU de su máquina.
"/gpu:0": La primera GPU de tu máquina
"/gpu:1": La segunda GPU de tu máquina
Para este ejemplo, estamos usando 2 GTX-980

In [2]:
import numpy as np
import tensorflow as tf
import datetime

In [3]:
#Logs de unidades de proceso
log_device_placement = True

#número de multiplicaciones a realizar
n = 10

In [ ]:
# Ejemplo: computa A^n + B^n en 2 GPUs

# Crear una matriz grande y aleatoria
A = np.random.rand(1e4, 1e4).astype('float32')
B = np.random.rand(1e4, 1e4).astype('float32')

# Crea un gráfico para almacenar los resultados
c1 = []
c2 = []

# Define el poder de la matriz
def matpow(M, n):
    if n < 1: #Casos abstractos en los que n < 1
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

In [6]:
# Cálculo en una sola GPU

with tf.device('/gpu:0'):
    a = tf.constant(A)
    b = tf.constant(B)
    # calcular A^n y B^n y guardar los resultados en c1
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))

with tf.device('/cpu:0'):
  sum = tf.add_n(c1) #Adición de todos los elementos en c1, es decir, A^n + B^n

t1_1 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    # Ejecuta la operación.
    sess.run(sum)
t2_1 = datetime.datetime.now()

In [7]:
# Multi GPU computing
# GPU:0 calcula A^n
with tf.device('/gpu:0'):
    #calcular A^n y guardar el resultado en c2
    a = tf.constant(A)
    c2.append(matpow(a, n))

#GPU:1 calcula B^n
with tf.device('/gpu:1'):
    #Calcula B^n y guarda el resultado en c2
    b = tf.constant(B)
    c2.append(matpow(b, n))

with tf.device('/cpu:0'):
  sum = tf.add_n(c2) #Adición de todos los elementos en c2, es decir, A^n + B^n

t1_2 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    # Ejecuta la operación.
    sess.run(sum)
t2_2 = datetime.datetime.now()

In [8]:
print "Single GPU computation time: " + str(t2_1-t1_1)
print "Multi GPU computation time: " + str(t2_2-t1_2)

Single GPU computation time: 0:00:11.833497
Multi GPU computation time: 0:00:07.085913
